In [1]:
import datetime 
import numpy as np
import mpmath as mp
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from sklearn.preprocessing import StandardScaler
%matplotlib qt

from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import os 
from os import listdir
from os.path import isfile, join

def get_directory(folder):
    files= [a for a in listdir(folder) if isfile(join(folder,a))]
    
    return files
import os 
from os import listdir
from os.path import isfile, join

"get directory 2.0"

def get_directory_contents(folder):
    items = os.listdir(folder)
    files = [item for item in items if os.path.isfile(os.path.join(folder, item))]
    folders = [item for item in items if os.path.isdir(os.path.join(folder, item))]
    
    return {'files': files, 'folders': folders}

In [2]:
from datetime import datetime 

def search_the_day(fecha):
    # Fecha del primer día del año
    primer_dia_del_ano = datetime(fecha.year, 1, 1)

    # Calcula la diferencia entre las dos fechas
    diferencia = fecha - primer_dia_del_ano

    # Obtiene el número de días transcurridos
    dias_transcurridos = diferencia.days+1
    #dias_transcurridos = diferencia.days
    
    return dias_transcurridos
    

## ITERATION

In [3]:
def ITERATION(datahistograms):
    a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]
    
    meanlist=[]
    stdlist=[]
    for i in np.arange(0,len(a1)):
        mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
        meanlist.append(mean)
        stdlist.append(std)
        
    return meanlist, stdlist, a1
    

In [4]:
def GlobalFunction(main_folder,hora1,hora2):
    
    list_of_files=get_directory_contents(main_folder)                 
    folders=list_of_files['folders']

    print(folders)

    TOTALDATA=[]

    for i in np.arange(0,len(folders)):
    #for i in np.arange(0,4):
        main_folder2=main_folder+'/'+str(folders[i])
        print(main_folder2)
        list_of_files2=get_directory_contents(main_folder2)                   
        files=list_of_files2['files']

        newlist=[]
        for item in files:                                        #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
            # Verificar si la cadena inicia con la letra "p"
            if item.startswith("p"):
                newlist.append(main_folder2+'/'+str(item))
        #print(newlist)

        #for j in np.arange(0,len(newlist)):

        'Iteracion for each hour of a day'

        list_of_data_per_day=[] #list with all the data per hour per day
        list_of_first_derivative_per_day=[]
        list_of_second_derivative_per_day=[]
        list_of_hours=[] 
        for j in np.arange(0,len(newlist)):
        #for j in np.arange(0,1):

            #-----------------------------------------------------------------------------------------------
            'Retain the date of the measurement'
            dato=pd.read_table(newlist[j]) 
            dato=dato.iloc[np.arange(2,3)][' PTS'][2]


            numeros = dato.split()
            numeros_enteros = [int(numero) for numero in numeros]
            numeros = numeros_enteros
            from datetime import datetime
            ano = 2000 + numeros[0]  
            mes = numeros[1]
            dia = numeros[2]
            hora = numeros[3]
            minuto = numeros[4]
            segundo = numeros[5]

            fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                        #print('fecha_hora',fecha_hora)
                        #print('fecha_hora',fecha_hora.hour)
            list_of_hours.append(fecha_hora)
            #-----------------------------------------------------------------------------------------------

            'Wind data'

            nombrearchivo = newlist[j] # Name of the file
            titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
            data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
            data=data.iloc[np.arange(1,len(data)),:] #delete the first row
            data=data.reset_index(drop=True)

            'Eliminate nans'
            data=data.astype(object)
            data = data.dropna()
            data=data.reset_index(drop=True)

            'Drop the rest of dataframes if there is more than 1'

            for k in np.arange(0,len(data)):
                if str(data['HT'][k])=='HT':
                    data=data.iloc[np.arange(0,k),:] 
                    break

            'Eliminar los 99999'
            for k in np.arange(0,len(data)):
                if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                    data.drop(k, axis=0, inplace=True)

            data=data.reset_index(drop=True)


            'moving average'
            data2=data[['HT','SPD','DIR']]

            window_size = 2  # Tamaño de la ventana de media móvil

            # Aplicar filtro de media móvil
            data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
            data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

            
            # let's see what happens with the first and second derivative CENTERED DERIVATIVE 
            
            first_ht=[]   
            first_derivative=[]
            second_ht=[]  
            second_derivative=[]
            
            data['HT']=data['HT'].astype(float)
            data['SPD']=data['SPD'].astype(float)
            
            for k in np.arange(0,len(data['HT'])-1):
                first_ht.append(data['HT'][k])
                derivative= ( (data['SPD'][k+1]-data['SPD'][k])/(data['HT'][k+1]-data['HT'][k]) )
                first_derivative.append(derivative)
            
            for k in np.arange(1,len(data['HT'])-1):
                second_ht.append(data['HT'][k])
                
                derivative= ( (data['SPD'][k+1]-2*data['SPD'][k]+data['SPD'][k-1]) / ((data['HT'][k+1]-data['HT'][k])**2) )
                second_derivative.append(derivative)
            
            
            d = {'HT_first': first_ht, 'SPD_first': first_derivative}
            FIRST_DERIVATIVE_TABLE = pd.DataFrame(data=d)
            
            dd = {'HT_second': second_ht, 'SPD_second': second_derivative}
            SECOND_DERIVATIVE_TABLE = pd.DataFrame(data=dd)
            
            #print(FIRST_DERIVATIVE_TABLE)
            #print(SECOND_DERIVATIVE_TABLE)
            
            
            list_of_data_per_day.append(data)
            list_of_first_derivative_per_day.append(FIRST_DERIVATIVE_TABLE)
            list_of_second_derivative_per_day.append(SECOND_DERIVATIVE_TABLE)

        TOTALDATA.append( [list_of_data_per_day, list_of_hours, list_of_first_derivative_per_day,list_of_second_derivative_per_day ] )
 
    #-------------------------------------------------------------------------------------------------
    datahistograms=[] 
    datahistograms1st=[] 
    datahistograms2nd=[] 
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=hora1 and TOTALDATA[k][1][l].hour<=hora2: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    def MeanAndStd(datahistograms,km1,km2):
        data=[]
        for i in np.arange(0,len(datahistograms)):
            if km1<=datahistograms[i][0]<=km2:
                data.append(datahistograms[i][1]) 
        mean=np.nanmean(data)
        std=np.nanstd(data)

        return  mean, std
    
    def ITERATION(datahistograms,datahistograms1st,datahistograms2nd):
        a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]

        meanlist=[]
        stdlist=[]
        
        meanlist1st=[]
        stdlist1st=[]
        
        meanlist2nd=[]
        stdlist2nd=[]
        
        for i in np.arange(0,len(a1)):
            mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
            meanlist.append(mean)
            stdlist.append(std)
            
        for i in np.arange(0,len(a1)-1):
            mean,std= MeanAndStd(datahistograms1st,a1[i][0],a1[i][1])
            meanlist1st.append(mean)
            stdlist1st.append(std)
            
        for i in np.arange(1,len(a1)-1):
            mean,std= MeanAndStd(datahistograms2nd,a1[i][0],a1[i][1])
            meanlist2nd.append(mean)
            stdlist2nd.append(std)    
            

        return meanlist, stdlist, a1,  meanlist1st,stdlist1st,  meanlist2nd,stdlist2nd
    #-------------------------------------------------------------------------------------------------
    means,deviations,kilometers , means1st,deviations1st,  means2nd,deviations2nd = ITERATION(datahistograms, datahistograms1st, datahistograms2nd)
    
    kilometersplot=[]
    kilometersplot2=[]
    kilometersplot3=[]
    for i in np.arange(0,len(kilometers)):
        kilometersplot.append( (kilometers[i][0]+kilometers[i][1])/2 )
    for i in np.arange(0,len(kilometers)-1):
        kilometersplot2.append( (kilometers[i][0]+kilometers[i][1])/2 )
    for i in np.arange(1,len(kilometers)-1):
        kilometersplot3.append( (kilometers[i][0]+kilometers[i][1])/2 )    


    plt.figure()
    plt.errorbar(kilometersplot, means, yerr=deviations, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s) ')
    
    plt.figure()
    plt.errorbar(kilometersplot2, means1st, yerr=deviations1st, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s km) ')
    plt.title('First derivative')
    
    plt.figure()
    plt.errorbar(kilometersplot3, means2nd, yerr=deviations2nd, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s km2) ')
    plt.title('Second derivative')
    
    return

In [5]:
def GlobalFunction2(main_folder):
    
    list_of_files=get_directory_contents(main_folder)                 
    folders=list_of_files['folders']

    print(folders)

    TOTALDATA=[]

    for i in np.arange(0,len(folders)):
    #for i in np.arange(0,4):
        main_folder2=main_folder+'/'+str(folders[i])
        print(main_folder2)
        list_of_files2=get_directory_contents(main_folder2)                   
        files=list_of_files2['files']

        newlist=[]
        for item in files:                                        #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
            # Verificar si la cadena inicia con la letra "p"
            if item.startswith("p"):
                newlist.append(main_folder2+'/'+str(item))
        #print(newlist)

        #for j in np.arange(0,len(newlist)):

        'Iteracion for each hour of a day'

        list_of_data_per_day=[] #list with all the data per hour per day
        list_of_first_derivative_per_day=[]
        list_of_second_derivative_per_day=[]
        list_of_hours=[] 
        for j in np.arange(0,len(newlist)):
        #for j in np.arange(0,1):

            #-----------------------------------------------------------------------------------------------
            'Retain the date of the measurement'
            dato=pd.read_table(newlist[j]) 
            dato=dato.iloc[np.arange(2,3)][' PTS'][2]


            numeros = dato.split()
            numeros_enteros = [int(numero) for numero in numeros]
            numeros = numeros_enteros
            from datetime import datetime
            ano = 2000 + numeros[0]  
            mes = numeros[1]
            dia = numeros[2]
            hora = numeros[3]
            minuto = numeros[4]
            segundo = numeros[5]

            fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                        #print('fecha_hora',fecha_hora)
                        #print('fecha_hora',fecha_hora.hour)
            list_of_hours.append(fecha_hora)
            #-----------------------------------------------------------------------------------------------

            'Wind data'

            nombrearchivo = newlist[j] # Name of the file
            titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
            data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
            data=data.iloc[np.arange(1,len(data)),:] #delete the first row
            data=data.reset_index(drop=True)

            'Eliminate nans'
            data=data.astype(object)
            data = data.dropna()
            data=data.reset_index(drop=True)

            'Drop the rest of dataframes if there is more than 1'

            for k in np.arange(0,len(data)):
                if str(data['HT'][k])=='HT':
                    data=data.iloc[np.arange(0,k),:] 
                    break

            'Eliminar los 99999'
            for k in np.arange(0,len(data)):
                if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                    data.drop(k, axis=0, inplace=True)

            data=data.reset_index(drop=True)


            'moving average'
            data2=data[['HT','SPD','DIR']]

            window_size = 2  # Tamaño de la ventana de media móvil

            # Aplicar filtro de media móvil
            data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
            data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

            
            # let's see what happens with the first and second derivative CENTERED DERIVATIVE 
            
            first_ht=[]   
            first_derivative=[]
            second_ht=[]  
            second_derivative=[]
            
            data['HT']=data['HT'].astype(float)
            data['SPD']=data['SPD'].astype(float)
            
            for k in np.arange(0,len(data['HT'])-1):
                first_ht.append(data['HT'][k])
                derivative= ( (data['SPD'][k+1]-data['SPD'][k])/(data['HT'][k+1]-data['HT'][k]) )
                first_derivative.append(derivative)
            
            for k in np.arange(1,len(data['HT'])-1):
                second_ht.append(data['HT'][k])
                
                derivative= ( (data['SPD'][k+1]-2*data['SPD'][k]+data['SPD'][k-1]) / ((data['HT'][k+1]-data['HT'][k])**2) )
                second_derivative.append(derivative)
            
            
            d = {'HT_first': first_ht, 'SPD_first': first_derivative}
            FIRST_DERIVATIVE_TABLE = pd.DataFrame(data=d)
            
            dd = {'HT_second': second_ht, 'SPD_second': second_derivative}
            SECOND_DERIVATIVE_TABLE = pd.DataFrame(data=dd)
            
            #print(FIRST_DERIVATIVE_TABLE)
            #print(SECOND_DERIVATIVE_TABLE)
            
            
            list_of_data_per_day.append(data)
            list_of_first_derivative_per_day.append(FIRST_DERIVATIVE_TABLE)
            list_of_second_derivative_per_day.append(SECOND_DERIVATIVE_TABLE)

        TOTALDATA.append( [list_of_data_per_day, list_of_hours, list_of_first_derivative_per_day,list_of_second_derivative_per_day ] )
 

    #-------------------------------------------------------------------------------------------------
    datahistograms1=[] 
    datahistograms1st1=[] 
    datahistograms2nd1=[] 
    
    datahistograms2=[] 
    datahistograms1st2=[] 
    datahistograms2nd2=[] 
    
    datahistograms3=[] 
    datahistograms1st3=[] 
    datahistograms2nd3=[] 
    
    datahistograms4=[] 
    datahistograms1st4=[] 
    datahistograms2nd4=[] 
    
    
    #Iteration for 4 time intervals
    
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=0 and TOTALDATA[k][1][l].hour<=6: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms1.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st1.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd1.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=6 and TOTALDATA[k][1][l].hour<=12: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms2.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st2.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd2.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=12 and TOTALDATA[k][1][l].hour<=18: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms3.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st3.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd3.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=18:    #and TOTALDATA[k][1][l].hour<=hora2: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms4.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st4.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd4.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    def MeanAndStd(datahistograms,km1,km2):
        data=[]
        for i in np.arange(0,len(datahistograms)):
            if km1<=datahistograms[i][0]<=km2:
                data.append(datahistograms[i][1]) 
        mean=np.nanmean(data)
        std=np.nanstd(data)

        return  mean, std
    
    def ITERATION(datahistograms,datahistograms1st,datahistograms2nd):
        a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]

        meanlist=[]
        stdlist=[]
        
        meanlist1st=[]
        stdlist1st=[]
        
        meanlist2nd=[]
        stdlist2nd=[]
        
        for i in np.arange(0,len(a1)):
            mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
            meanlist.append(mean)
            stdlist.append(std)
            
        for i in np.arange(0,len(a1)-1):
            mean,std= MeanAndStd(datahistograms1st,a1[i][0],a1[i][1])
            meanlist1st.append(mean)
            stdlist1st.append(std)
            
        for i in np.arange(1,len(a1)-1):
            mean,std= MeanAndStd(datahistograms2nd,a1[i][0],a1[i][1])
            meanlist2nd.append(mean)
            stdlist2nd.append(std)    
            

        return meanlist, stdlist, a1,  meanlist1st,stdlist1st,  meanlist2nd,stdlist2nd
    #-------------------------------------------------------------------------------------------------
    means1,deviations1,kilometers1 , means1st1,deviations1st1,  means2nd1,deviations2nd1 = ITERATION(datahistograms1, datahistograms1st1, datahistograms2nd1)
    means2,deviations2,kilometers2 , means1st2,deviations1st2,  means2nd2,deviations2nd2 = ITERATION(datahistograms2, datahistograms1st2, datahistograms2nd2)
    means3,deviations3,kilometers3 , means1st3,deviations1st3,  means2nd3,deviations2nd3 = ITERATION(datahistograms3, datahistograms1st3, datahistograms2nd3)
    means4,deviations4,kilometers4 , means1st4,deviations1st4,  means2nd4,deviations2nd4 = ITERATION(datahistograms4, datahistograms1st4, datahistograms2nd4)
    
    
    kilometersplot=[]
    kilometersplot2=[]
    kilometersplot3=[]
    for i in np.arange(0,len(kilometers1)):
        kilometersplot.append( (kilometers1[i][0]+kilometers1[i][1])/2 )
    for i in np.arange(0,len(kilometers1)-1):
        kilometersplot2.append( (kilometers1[i][0]+kilometers1[i][1])/2 )
    for i in np.arange(1,len(kilometers1)-1):
        kilometersplot3.append( (kilometers1[i][0]+kilometers1[i][1])/2 )    


    plt.figure()
    plt.style.use('tableau-colorblind10')
    plt.errorbar(kilometersplot, means1, yerr=deviations1, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(kilometersplot, means2, yerr=deviations2, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(kilometersplot, means3, yerr=deviations3, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(kilometersplot, means4, yerr=deviations4, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    plt.legend()
    #plt.ylim(0,20)
    plt.xlabel('Altura (km)',fontsize=13)
    plt.ylabel('V  (m/s) ',fontsize=13)
    
    plt.figure()
    plt.errorbar(kilometersplot2, means1st1, yerr=deviations1st1, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(kilometersplot2, means1st2, yerr=deviations1st2, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(kilometersplot2, means1st3, yerr=deviations1st3, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(kilometersplot2, means1st4, yerr=deviations1st4, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    plt.legend()
    #plt.ylim(-25,25)
    plt.xlabel('Altura (km)', fontsize=13)
    plt.ylabel(r'$\nabla V$'+' (m/s km)', fontsize=13 )
    #plt.title('First derivative')
    
    plt.figure()
    plt.errorbar(kilometersplot3, means2nd1, yerr=deviations2nd1, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(kilometersplot3, means2nd2, yerr=deviations2nd2, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(kilometersplot3, means2nd3, yerr=deviations2nd3, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(kilometersplot3, means2nd4, yerr=deviations2nd4, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    plt.legend()
    #plt.ylim(-220,220)
    plt.xlabel('Altura (km)', fontsize=13)
    plt.ylabel(r'$\nabla^2 V$'+' (m/s km2)', fontsize=13 )
    #plt.title('Second derivative')
    
    return

In [9]:
def GlobalFunction3(main_folder):
    
    list_of_files=get_directory_contents(main_folder)                 
    folders=list_of_files['folders']

    print(folders)

    TOTALDATA=[]

    for i in np.arange(0,len(folders)):
    #for i in np.arange(0,10):
        main_folder2=main_folder+'/'+str(folders[i])
        print(main_folder2)
        list_of_files2=get_directory_contents(main_folder2)                   
        files=list_of_files2['files']

        newlist=[]
        for item in files:                                        #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
            # Verificar si la cadena inicia con la letra "p"
            if item.startswith("p"):
                newlist.append(main_folder2+'/'+str(item))
        #print(newlist)

        #for j in np.arange(0,len(newlist)):

        'Iteracion for each hour of a day'

        list_of_data_per_day=[] #list with all the data per hour per day
        list_of_first_derivative_per_day=[]
        list_of_second_derivative_per_day=[]
        list_of_hours=[] 
        for j in np.arange(0,len(newlist)):
        #for j in np.arange(0,1):

            #-----------------------------------------------------------------------------------------------
            'Retain the date of the measurement'
            dato=pd.read_table(newlist[j]) 
            dato=dato.iloc[np.arange(2,3)][' PTS'][2]


            numeros = dato.split()
            numeros_enteros = [int(numero) for numero in numeros]
            numeros = numeros_enteros
            from datetime import datetime
            ano = 2000 + numeros[0]  
            mes = numeros[1]
            dia = numeros[2]
            hora = numeros[3]
            minuto = numeros[4]
            segundo = numeros[5]

            fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                        #print('fecha_hora',fecha_hora)
                        #print('fecha_hora',fecha_hora.hour)
            list_of_hours.append(fecha_hora)
            #-----------------------------------------------------------------------------------------------

            'Wind data'

            nombrearchivo = newlist[j] # Name of the file
            titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
            data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
            data=data.iloc[np.arange(1,len(data)),:] #delete the first row
            data=data.reset_index(drop=True)

            'Eliminate nans'
            data=data.astype(object)
            data = data.dropna()
            data=data.reset_index(drop=True)

            'Drop the rest of dataframes if there is more than 1'

            for k in np.arange(0,len(data)):
                if str(data['HT'][k])=='HT':
                    data=data.iloc[np.arange(0,k),:] 
                    break

            'Eliminar los 99999'
            for k in np.arange(0,len(data)):
                if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                    data.drop(k, axis=0, inplace=True)

            data=data.reset_index(drop=True)


            'moving average'
            data2=data[['HT','SPD','DIR']]

            window_size = 2  # Tamaño de la ventana de media móvil

            # Aplicar filtro de media móvil
            data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
            data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

            
            # let's see what happens with the first and second derivative CENTERED DERIVATIVE 
            
            first_ht=[]   
            first_derivative=[]
            second_ht=[]  
            second_derivative=[]
            
            data['HT']=data['HT'].astype(float)
            data['SPD']=data['SPD'].astype(float)
            
            for k in np.arange(0,len(data['HT'])-1):
                first_ht.append(data['HT'][k])
                derivative= ( (data['SPD'][k+1]-data['SPD'][k])/(data['HT'][k+1]-data['HT'][k]) )
                first_derivative.append(derivative)
            
            for k in np.arange(1,len(data['HT'])-1):
                second_ht.append(data['HT'][k])
                
                derivative= ( (data['SPD'][k+1]-2*data['SPD'][k]+data['SPD'][k-1]) / ((data['HT'][k+1]-data['HT'][k])**2) )
                second_derivative.append(derivative)
            
            
            d = {'HT_first': first_ht, 'SPD_first': first_derivative}
            FIRST_DERIVATIVE_TABLE = pd.DataFrame(data=d)
            
            dd = {'HT_second': second_ht, 'SPD_second': second_derivative}
            SECOND_DERIVATIVE_TABLE = pd.DataFrame(data=dd)
            
            #print(FIRST_DERIVATIVE_TABLE)
            #print(SECOND_DERIVATIVE_TABLE)
            
            
            list_of_data_per_day.append(data)
            list_of_first_derivative_per_day.append(FIRST_DERIVATIVE_TABLE)
            list_of_second_derivative_per_day.append(SECOND_DERIVATIVE_TABLE)

        TOTALDATA.append( [list_of_data_per_day, list_of_hours, list_of_first_derivative_per_day,list_of_second_derivative_per_day ] )
 

    #-------------------------------------------------------------------------------------------------
    datahistograms1=[] 
    datahistograms1st1=[] 
    datahistograms2nd1=[] 
    
    datahistograms2=[] 
    datahistograms1st2=[] 
    datahistograms2nd2=[] 
    
    datahistograms3=[] 
    datahistograms1st3=[] 
    datahistograms2nd3=[] 
    
    datahistograms4=[] 
    datahistograms1st4=[] 
    datahistograms2nd4=[] 
    
    
    #Iteration for 4 time intervals
    
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=0 and TOTALDATA[k][1][l].hour<6: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms1.append( [ [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z])] , TOTALDATA[k][1][l] ])
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st1.append( [ [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z])] , TOTALDATA[k][1][l]  ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd1.append( [ [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z])], TOTALDATA[k][1][l]  ]  )    
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=6 and TOTALDATA[k][1][l].hour<12: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms2.append( [ [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z])] , TOTALDATA[k][1][l] ]) 
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st2.append( [ [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ]  , TOTALDATA[k][1][l]  ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd2.append( [ [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z])], TOTALDATA[k][1][l]  ]  )  
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=12 and TOTALDATA[k][1][l].hour<18: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms3.append(  [ [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z])], TOTALDATA[k][1][l] ]) 
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st3.append(  [ [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ]  , TOTALDATA[k][1][l]  ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd3.append( [ [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z])] , TOTALDATA[k][1][l]  ] )    
    #-------------------------------------------------------------------------------------------------
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=18:    #and TOTALDATA[k][1][l].hour<=hora2: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms4.append( [ [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z])], TOTALDATA[k][1][l] ])  
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st4.append( [ [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] , TOTALDATA[k][1][l]  ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd4.append( [ [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z])] , TOTALDATA[k][1][l]  ] )    
   

    #ITERATIONS FOR EVERY MONTH 
    
    histogram1month1=[] 
    histogram1month2=[] 
    histogram1month3=[] 
    histogram1month4=[] 
    histogram1month5=[] 
    histogram1month6=[] 
    histogram1month7=[] 
    histogram1month8=[] 
    histogram1month9=[] 
    histogram1month10=[] 
    histogram1month11=[] 
    histogram1month12=[] 
    
    histogram1stmonth1=[] 
    histogram1stmonth2=[] 
    histogram1stmonth3=[] 
    histogram1stmonth4=[] 
    histogram1stmonth5=[] 
    histogram1stmonth6=[] 
    histogram1stmonth7=[] 
    histogram1stmonth8=[] 
    histogram1stmonth9=[] 
    histogram1stmonth10=[] 
    histogram1stmonth11=[] 
    histogram1stmonth12=[] 
    
    
    for k in np.arange(0,len(datahistograms1)):   
        if datahistograms1[k][1].month==1:
            histogram1month1.append(datahistograms1[k])
        if datahistograms1[k][1].month==2:
            histogram1month2.append(datahistograms1[k])
        if datahistograms1[k][1].month==3:
            histogram1month3.append(datahistograms1[k])
        if datahistograms1[k][1].month==4:
            histogram1month4.append(datahistograms1[k])
        if datahistograms1[k][1].month==5:
            histogram1month5.append(datahistograms1[k])
        if datahistograms1[k][1].month==6:
            histogram1month6.append(datahistograms1[k])
        if datahistograms1[k][1].month==7:
            histogram1month7.append(datahistograms1[k])
        if datahistograms1[k][1].month==8:
            histogram1month8.append(datahistograms1[k])
        if datahistograms1[k][1].month==9:
            histogram1month9.append(datahistograms1[k])
        if datahistograms1[k][1].month==10:
            histogram1month10.append(datahistograms1[k])    
        if datahistograms1[k][1].month==11:
            histogram1month11.append(datahistograms1[k])
        if datahistograms1[k][1].month==12:
            histogram1month12.append(datahistograms1[k])
       
    for k in np.arange(0,len(datahistograms1st1)):   
        if datahistograms1st1[k][1].month==1:
            histogram1stmonth1.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==2:
            histogram1stmonth2.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==3:
            histogram1stmonth3.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==4:
            histogram1stmonth4.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==5:
            histogram1stmonth5.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==6:
            histogram1stmonth6.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==7:
            histogram1stmonth7.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==8:
            histogram1stmonth8.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==9:
            histogram1stmonth9.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==10:
            histogram1stmonth10.append(datahistograms1st1[k])    
        if datahistograms1st1[k][1].month==11:
            histogram1stmonth11.append(datahistograms1st1[k])
        if datahistograms1st1[k][1].month==12:
            histogram1stmonth12.append(datahistograms1st1[k])
            
            
    histogram2month1=[] 
    histogram2month2=[] 
    histogram2month3=[] 
    histogram2month4=[] 
    histogram2month5=[] 
    histogram2month6=[] 
    histogram2month7=[] 
    histogram2month8=[] 
    histogram2month9=[] 
    histogram2month10=[] 
    histogram2month11=[] 
    histogram2month12=[]
     
    histogram1st2month1=[] 
    histogram1st2month2=[] 
    histogram1st2month3=[] 
    histogram1st2month4=[] 
    histogram1st2month5=[] 
    histogram1st2month6=[] 
    histogram1st2month7=[] 
    histogram1st2month8=[] 
    histogram1st2month9=[] 
    histogram1st2month10=[] 
    histogram1st2month11=[] 
    histogram1st2month12=[] 
    for k in np.arange(0,len(datahistograms2)):   
        if datahistograms2[k][1].month==1:
            histogram2month1.append(datahistograms2[k])
        if datahistograms2[k][1].month==2:
            histogram2month2.append(datahistograms2[k])
        if datahistograms2[k][1].month==3:
            histogram2month3.append(datahistograms2[k])
        if datahistograms2[k][1].month==4:
            histogram2month4.append(datahistograms2[k])
        if datahistograms2[k][1].month==5:
            histogram2month5.append(datahistograms2[k])
        if datahistograms2[k][1].month==6:
            histogram2month6.append(datahistograms2[k])
        if datahistograms2[k][1].month==7:
            histogram2month7.append(datahistograms2[k])
        if datahistograms2[k][1].month==8:
            histogram2month8.append(datahistograms2[k])
        if datahistograms2[k][1].month==9:
            histogram2month9.append(datahistograms2[k])
        if datahistograms2[k][1].month==10:
            histogram2month10.append(datahistograms2[k])    
        if datahistograms2[k][1].month==11:
            histogram2month11.append(datahistograms2[k])
        if datahistograms2[k][1].month==12:
            histogram2month12.append(datahistograms2[k])
       
    for k in np.arange(0,len(datahistograms1st2)):   
        if datahistograms1st2[k][1].month==1:
            histogram1st2month1.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==2:
            histogram1st2month2.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==3:
            histogram1st2month3.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==4:
            histogram1st2month4.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==5:
            histogram1st2month5.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==6:
            histogram1st2month6.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==7:
            histogram1st2month7.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==8:
            histogram1st2month8.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==9:
            histogram1st2month9.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==10:
            histogram1st2month10.append(datahistograms1st2[k])    
        if datahistograms1st2[k][1].month==11:
            histogram1st2month11.append(datahistograms1st2[k])
        if datahistograms1st2[k][1].month==12:
            histogram1st2month12.append(datahistograms1st2[k])  
    
    
    histogram3month1=[] 
    histogram3month2=[] 
    histogram3month3=[] 
    histogram3month4=[] 
    histogram3month5=[] 
    histogram3month6=[] 
    histogram3month7=[] 
    histogram3month8=[] 
    histogram3month9=[] 
    histogram3month10=[] 
    histogram3month11=[] 
    histogram3month12=[]
     
    histogram1st3month1=[] 
    histogram1st3month2=[] 
    histogram1st3month3=[] 
    histogram1st3month4=[] 
    histogram1st3month5=[] 
    histogram1st3month6=[] 
    histogram1st3month7=[] 
    histogram1st3month8=[] 
    histogram1st3month9=[] 
    histogram1st3month10=[] 
    histogram1st3month11=[] 
    histogram1st3month12=[] 
    for k in np.arange(0,len(datahistograms3)):   
        if datahistograms3[k][1].month==1:
            histogram3month1.append(datahistograms3[k])
        if datahistograms3[k][1].month==2:
            histogram3month2.append(datahistograms3[k])
        if datahistograms3[k][1].month==3:
            histogram3month3.append(datahistograms3[k])
        if datahistograms3[k][1].month==4:
            histogram3month4.append(datahistograms3[k])
        if datahistograms3[k][1].month==5:
            histogram3month5.append(datahistograms3[k])
        if datahistograms3[k][1].month==6:
            histogram3month6.append(datahistograms3[k])
        if datahistograms3[k][1].month==7:
            histogram3month7.append(datahistograms3[k])
        if datahistograms3[k][1].month==8:
            histogram3month8.append(datahistograms3[k])
        if datahistograms3[k][1].month==9:
            histogram3month9.append(datahistograms3[k])
        if datahistograms3[k][1].month==10:
            histogram3month10.append(datahistograms3[k])    
        if datahistograms3[k][1].month==11:
            histogram3month11.append(datahistograms3[k])
        if datahistograms3[k][1].month==12:
            histogram3month12.append(datahistograms3[k])
       
    for k in np.arange(0,len(datahistograms1st3)):   
        if datahistograms1st3[k][1].month==1:
            histogram1st3month1.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==2:
            histogram1st3month2.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==3:
            histogram1st3month3.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==4:
            histogram1st3month4.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==5:
            histogram1st3month5.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==6:
            histogram1st3month6.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==7:
            histogram1st3month7.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==8:
            histogram1st3month8.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==9:
            histogram1st3month9.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==10:
            histogram1st3month10.append(datahistograms1st3[k])    
        if datahistograms1st3[k][1].month==11:
            histogram1st3month11.append(datahistograms1st3[k])
        if datahistograms1st3[k][1].month==12:
            histogram1st3month12.append(datahistograms1st3[k])  
    
    histogram4month1=[] 
    histogram4month2=[] 
    histogram4month3=[] 
    histogram4month4=[] 
    histogram4month5=[] 
    histogram4month6=[] 
    histogram4month7=[] 
    histogram4month8=[] 
    histogram4month9=[] 
    histogram4month10=[] 
    histogram4month11=[] 
    histogram4month12=[]
     
    histogram1st4month1=[] 
    histogram1st4month2=[] 
    histogram1st4month3=[] 
    histogram1st4month4=[] 
    histogram1st4month5=[] 
    histogram1st4month6=[] 
    histogram1st4month7=[] 
    histogram1st4month8=[] 
    histogram1st4month9=[] 
    histogram1st4month10=[] 
    histogram1st4month11=[] 
    histogram1st4month12=[] 
    for k in np.arange(0,len(datahistograms4)):   
        if datahistograms4[k][1].month==1:
            histogram4month1.append(datahistograms4[k])
        if datahistograms4[k][1].month==2:
            histogram4month2.append(datahistograms4[k])
        if datahistograms4[k][1].month==3:
            histogram4month3.append(datahistograms4[k])
        if datahistograms4[k][1].month==4:
            histogram4month4.append(datahistograms4[k])
        if datahistograms4[k][1].month==5:
            histogram4month5.append(datahistograms4[k])
        if datahistograms4[k][1].month==6:
            histogram4month6.append(datahistograms4[k])
        if datahistograms4[k][1].month==7:
            histogram4month7.append(datahistograms4[k])
        if datahistograms4[k][1].month==8:
            histogram4month8.append(datahistograms4[k])
        if datahistograms4[k][1].month==9:
            histogram4month9.append(datahistograms4[k])
        if datahistograms4[k][1].month==10:
            histogram4month10.append(datahistograms4[k])    
        if datahistograms4[k][1].month==11:
            histogram4month11.append(datahistograms4[k])
        if datahistograms4[k][1].month==12:
            histogram4month12.append(datahistograms4[k])
       
    for k in np.arange(0,len(datahistograms1st4)):   
        if datahistograms1st4[k][1].month==1:
            histogram1st4month1.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==2:
            histogram1st4month2.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==3:
            histogram1st4month3.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==4:
            histogram1st4month4.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==5:
            histogram1st4month5.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==6:
            histogram1st4month6.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==7:
            histogram1st4month7.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==8:
            histogram1st4month8.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==9:
            histogram1st4month9.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==10:
            histogram1st4month10.append(datahistograms1st4[k])    
        if datahistograms1st4[k][1].month==11:
            histogram1st4month11.append(datahistograms1st4[k])
        if datahistograms1st4[k][1].month==12:
            histogram1st4month12.append(datahistograms1st4[k])  
    
    
    #-------------------------------------------------------------------------------------------------
    def MeanAndStd(datahistograms,km1,km2):
        data=[]
        for i in np.arange(0,len(datahistograms)):
            if km1<=datahistograms[i][0][0]<=km2:
                data.append(datahistograms[i][0][1]) 
        mean=np.nanmean(data)
        std=np.nanstd(data)

        return  mean, std
    
    def ITERATION(datahistograms,datahistograms1st):
        a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]

        meanlist=[]
        stdlist=[]
        
        meanlist1st=[]
        stdlist1st=[]
        
        
        for i in np.arange(0,len(a1)):
            mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
            meanlist.append(mean)
            stdlist.append(std)
            
        for i in np.arange(0,len(a1)-1):
            mean,std= MeanAndStd(datahistograms1st,a1[i][0],a1[i][1])
            meanlist1st.append(mean)
            stdlist1st.append(std)
              
            

        return meanlist, stdlist, a1,  meanlist1st,stdlist1st
    #-------------------------------------------------------------------------------------------------
    datapermonth1=[ histogram1month1,histogram1month2,histogram1month3,histogram1month4,histogram1month5,histogram1month6,histogram1month7,histogram1month8,histogram1month9,histogram1month10,histogram1month11,histogram1month12] 
    datapermonth1st1=[ histogram1stmonth1,histogram1stmonth2,histogram1stmonth3,histogram1stmonth4,histogram1stmonth5,histogram1stmonth6,histogram1stmonth7,histogram1stmonth8,histogram1stmonth9,histogram1stmonth10,histogram1stmonth11,histogram1stmonth12] 
    
    datapermonth2=[ histogram2month1,histogram2month2,histogram2month3,histogram2month4,histogram2month5,histogram2month6,histogram2month7,histogram2month8,histogram2month9,histogram2month10,histogram2month11,histogram2month12] 
    datapermonth1st2=[ histogram1st2month1,histogram1st2month2,histogram1st2month3,histogram1st2month4,histogram1st2month5,histogram1st2month6,histogram1st2month7,histogram1st2month8,histogram1st2month9,histogram1st2month10,histogram1st2month11,histogram1st2month12] 
    
    datapermonth3=[ histogram3month1,histogram3month2,histogram3month3,histogram3month4,histogram3month5,histogram3month6,histogram3month7,histogram3month8,histogram3month9,histogram3month10,histogram3month11,histogram3month12] 
    datapermonth1st3=[ histogram1st3month1,histogram1st3month2,histogram1st3month3,histogram1st3month4,histogram1st3month5,histogram1st3month6,histogram1st3month7,histogram1st3month8,histogram1st3month9,histogram1st3month10,histogram1st3month11,histogram1st3month12] 
    
    datapermonth4=[ histogram4month1,histogram4month2,histogram4month3,histogram4month4,histogram4month5,histogram4month6,histogram4month7,histogram4month8,histogram4month9,histogram4month10,histogram4month11,histogram4month12] 
    datapermonth1st4=[ histogram1st4month1,histogram1st4month2,histogram1st4month3,histogram1st4month4,histogram1st4month5,histogram1st4month6,histogram1st4month7,histogram1st4month8,histogram1st4month9,histogram1st4month10,histogram1st4month11,histogram1st4month12] 
    
    #-------------------------------------------------------------------------------------------------------------------------------
    listfofmeans1=[]
    listfodeviations1=[]
    listfofmeans1st1=[]
    listfofdeviations1st1=[]
    for k in np.arange(0,len(datapermonth1)):
        a1=datapermonth1[k]
        a2=datapermonth1st1[k]
        means1,deviations1,kilometers1 , means1st1,deviations1st1 = ITERATION(a1, a2)
        listfofmeans1.append(means1)
        listfodeviations1.append(deviations1)
        listfofmeans1st1.append(means1st1)
        listfofdeviations1st1.append(deviations1st1)
    
    listfofmeans2=[]
    listfodeviations2=[]
    listfofmeans1st2=[]
    listfofdeviations1st2=[]
    for k in np.arange(0,len(datapermonth2)):
        a1=datapermonth2[k]
        a2=datapermonth1st2[k]
        means1,deviations1,kilometers1 , means1st1,deviations1st1 = ITERATION(a1, a2)
        listfofmeans2.append(means1)
        listfodeviations2.append(deviations1)
        listfofmeans1st2.append(means1st1)
        listfofdeviations1st2.append(deviations1st1)
        
    listfofmeans3=[]
    listfodeviations3=[]
    listfofmeans1st3=[]
    listfofdeviations1st3=[]
    for k in np.arange(0,len(datapermonth3)):
        a1=datapermonth3[k]
        a2=datapermonth1st3[k]
        means1,deviations1,kilometers1 , means1st1,deviations1st1 = ITERATION(a1, a2)
        listfofmeans3.append(means1)
        listfodeviations3.append(deviations1)
        listfofmeans1st3.append(means1st1)
        listfofdeviations1st3.append(deviations1st1)    
    
    listfofmeans4=[]
    listfodeviations4=[]
    listfofmeans1st4=[]
    listfofdeviations1st4=[]
    for k in np.arange(0,len(datapermonth4)):
        a1=datapermonth4[k]
        a2=datapermonth1st4[k]
        means1,deviations1,kilometers1 , means1st1,deviations1st1 = ITERATION(a1, a2)
        listfofmeans4.append(means1)
        listfodeviations4.append(deviations1)
        listfofmeans1st4.append(means1st1)
        listfofdeviations1st4.append(deviations1st1)    
    
    #----------------------------------------------------------------------------------------------------------------------------------------
    
    dataplotkm1=[]
    dataplotkm2=[]
    dataplotkm3=[]
    dataplotkm4=[]
    dataplotkm5=[]
    dataplotkm6=[]
    dataplotkm7=[]
    dataplotkm8=[]
    dataplotkm9=[]
    dataplotkm10=[]
    dataplotkm11=[]
    dataplotkm12=[]
    dataplotkm13=[]
    
    dataplotkm1error=[]
    dataplotkm2error=[]
    dataplotkm3error=[]
    dataplotkm4error=[]
    dataplotkm5error=[]
    dataplotkm6error=[]
    dataplotkm7error=[]
    dataplotkm8error=[]
    dataplotkm9error=[]
    dataplotkm10error=[]
    dataplotkm11error=[]
    dataplotkm12error=[]
    dataplotkm13error=[]
    
    
    dataplot2km1=[]
    dataplot2km2=[]
    dataplot2km3=[]
    dataplot2km4=[]
    dataplot2km5=[]
    dataplot2km6=[]
    dataplot2km7=[]
    dataplot2km8=[]
    dataplot2km9=[]
    dataplot2km10=[]
    dataplot2km11=[]
    dataplot2km12=[]
    dataplot2km13=[]
    
    dataplot2km1error=[]
    dataplot2km2error=[]
    dataplot2km3error=[]
    dataplot2km4error=[]
    dataplot2km5error=[]
    dataplot2km6error=[]
    dataplot2km7error=[]
    dataplot2km8error=[]
    dataplot2km9error=[]
    dataplot2km10error=[]
    dataplot2km11error=[]
    dataplot2km12error=[]
    dataplot2km13error=[]
    
    
    dataplot3km1=[]
    dataplot3km2=[]
    dataplot3km3=[]
    dataplot3km4=[]
    dataplot3km5=[]
    dataplot3km6=[]
    dataplot3km7=[]
    dataplot3km8=[]
    dataplot3km9=[]
    dataplot3km10=[]
    dataplot3km11=[]
    dataplot3km12=[]
    dataplot3km13=[]
    
    dataplot3km1error=[]
    dataplot3km2error=[]
    dataplot3km3error=[]
    dataplot3km4error=[]
    dataplot3km5error=[]
    dataplot3km6error=[]
    dataplot3km7error=[]
    dataplot3km8error=[]
    dataplot3km9error=[]
    dataplot3km10error=[]
    dataplot3km11error=[]
    dataplot3km12error=[]
    dataplot3km13error=[]
    
    
    dataplot4km1=[]
    dataplot4km2=[]
    dataplot4km3=[]
    dataplot4km4=[]
    dataplot4km5=[]
    dataplot4km6=[]
    dataplot4km7=[]
    dataplot4km8=[]
    dataplot4km9=[]
    dataplot4km10=[]
    dataplot4km11=[]
    dataplot4km12=[]
    dataplot4km13=[]
    
    dataplot4km1error=[]
    dataplot4km2error=[]
    dataplot4km3error=[]
    dataplot4km4error=[]
    dataplot4km5error=[]
    dataplot4km6error=[]
    dataplot4km7error=[]
    dataplot4km8error=[]
    dataplot4km9error=[]
    dataplot4km10error=[]
    dataplot4km11error=[]
    dataplot4km12error=[]
    dataplot4km13error=[]
    
    for k in np.arange(0,len(listfofmeans1)):
        dataplotkm1.append(listfofmeans1[k][0])
        dataplotkm2.append(listfofmeans1[k][1])
        dataplotkm3.append(listfofmeans1[k][2])
        dataplotkm4.append(listfofmeans1[k][3])
        dataplotkm5.append(listfofmeans1[k][4])
        dataplotkm6.append(listfofmeans1[k][5])
        dataplotkm7.append(listfofmeans1[k][6])
        dataplotkm8.append(listfofmeans1[k][7])
        dataplotkm9.append(listfofmeans1[k][8])
        dataplotkm10.append(listfofmeans1[k][9])
        dataplotkm11.append(listfofmeans1[k][10])
        dataplotkm12.append(listfofmeans1[k][11])
        dataplotkm13.append(listfofmeans1[k][12])
        
        dataplotkm1error.append(listfodeviations1[k][0])
        dataplotkm2error.append(listfodeviations1[k][1])
        dataplotkm3error.append(listfodeviations1[k][2])
        dataplotkm4error.append(listfodeviations1[k][3])
        dataplotkm5error.append(listfodeviations1[k][4])
        dataplotkm6error.append(listfodeviations1[k][5])
        dataplotkm7error.append(listfodeviations1[k][6])
        dataplotkm8error.append(listfodeviations1[k][7])
        dataplotkm9error.append(listfodeviations1[k][8])
        dataplotkm10error.append(listfodeviations1[k][9])
        dataplotkm11error.append(listfodeviations1[k][10])
        dataplotkm12error.append(listfodeviations1[k][11])
        dataplotkm13error.append(listfodeviations1[k][12])
        
        dataplot2km1.append(listfofmeans2[k][0])
        dataplot2km2.append(listfofmeans2[k][1])
        dataplot2km3.append(listfofmeans2[k][2])
        dataplot2km4.append(listfofmeans2[k][3])
        dataplot2km5.append(listfofmeans2[k][4])
        dataplot2km6.append(listfofmeans2[k][5])
        dataplot2km7.append(listfofmeans2[k][6])
        dataplot2km8.append(listfofmeans2[k][7])
        dataplot2km9.append(listfofmeans2[k][8])
        dataplot2km10.append(listfofmeans2[k][9])
        dataplot2km11.append(listfofmeans2[k][10])
        dataplot2km12.append(listfofmeans2[k][11])
        dataplot2km13.append(listfofmeans2[k][12])
        
        dataplot2km1error.append(listfodeviations2[k][0])
        dataplot2km2error.append(listfodeviations2[k][1])
        dataplot2km3error.append(listfodeviations2[k][2])
        dataplot2km4error.append(listfodeviations2[k][3])
        dataplot2km5error.append(listfodeviations2[k][4])
        dataplot2km6error.append(listfodeviations2[k][5])
        dataplot2km7error.append(listfodeviations2[k][6])
        dataplot2km8error.append(listfodeviations2[k][7])
        dataplot2km9error.append(listfodeviations2[k][8])
        dataplot2km10error.append(listfodeviations2[k][9])
        dataplot2km11error.append(listfodeviations2[k][10])
        dataplot2km12error.append(listfodeviations2[k][11])
        dataplot2km13error.append(listfodeviations2[k][12])
        
        
        dataplot3km1.append(listfofmeans3[k][0])
        dataplot3km2.append(listfofmeans3[k][1])
        dataplot3km3.append(listfofmeans3[k][2])
        dataplot3km4.append(listfofmeans3[k][3])
        dataplot3km5.append(listfofmeans3[k][4])
        dataplot3km6.append(listfofmeans3[k][5])
        dataplot3km7.append(listfofmeans3[k][6])
        dataplot3km8.append(listfofmeans3[k][7])
        dataplot3km9.append(listfofmeans3[k][8])
        dataplot3km10.append(listfofmeans3[k][9])
        dataplot3km11.append(listfofmeans3[k][10])
        dataplot3km12.append(listfofmeans3[k][11])
        dataplot3km13.append(listfofmeans3[k][12])
        
        dataplot3km1error.append(listfodeviations3[k][0])
        dataplot3km2error.append(listfodeviations3[k][1])
        dataplot3km3error.append(listfodeviations3[k][2])
        dataplot3km4error.append(listfodeviations3[k][3])
        dataplot3km5error.append(listfodeviations3[k][4])
        dataplot3km6error.append(listfodeviations3[k][5])
        dataplot3km7error.append(listfodeviations3[k][6])
        dataplot3km8error.append(listfodeviations3[k][7])
        dataplot3km9error.append(listfodeviations3[k][8])
        dataplot3km10error.append(listfodeviations3[k][9])
        dataplot3km11error.append(listfodeviations3[k][10])
        dataplot3km12error.append(listfodeviations3[k][11])
        dataplot3km13error.append(listfodeviations3[k][12])
        
        
        dataplot4km1.append(listfofmeans4[k][0])
        dataplot4km2.append(listfofmeans4[k][1])
        dataplot4km3.append(listfofmeans4[k][2])
        dataplot4km4.append(listfofmeans4[k][3])
        dataplot4km5.append(listfofmeans4[k][4])
        dataplot4km6.append(listfofmeans4[k][5])
        dataplot4km7.append(listfofmeans4[k][6])
        dataplot4km8.append(listfofmeans4[k][7])
        dataplot4km9.append(listfofmeans4[k][8])
        dataplot4km10.append(listfofmeans4[k][9])
        dataplot4km11.append(listfofmeans4[k][10])
        dataplot4km12.append(listfofmeans4[k][11])
        dataplot4km13.append(listfofmeans4[k][12])
        
        dataplot4km1error.append(listfodeviations4[k][0])
        dataplot4km2error.append(listfodeviations4[k][1])
        dataplot4km3error.append(listfodeviations4[k][2])
        dataplot4km4error.append(listfodeviations4[k][3])
        dataplot4km5error.append(listfodeviations4[k][4])
        dataplot4km6error.append(listfodeviations4[k][5])
        dataplot4km7error.append(listfodeviations4[k][6])
        dataplot4km8error.append(listfodeviations4[k][7])
        dataplot4km9error.append(listfodeviations4[k][8])
        dataplot4km10error.append(listfodeviations4[k][9])
        dataplot4km11error.append(listfodeviations4[k][10])
        dataplot4km12error.append(listfodeviations4[k][11])
        dataplot4km13error.append(listfodeviations4[k][12])
            
               
      
 
               
            
    #PLOT
    ylim1=0
    ylim2=25
    
    
    plt.figure()
    x=np.arange(0,12)
    

    ax7=plt.subplot(616)

    figsize=(20,4) 
    plt.errorbar(x, dataplotkm6, yerr=dataplotkm6error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km6, yerr=dataplot2km6error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km6, yerr=dataplot3km6error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km6, yerr=dataplot4km6error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    #plt.text(0.01, 0.75, "E)", fontweight="bold", fontsize=13 ,transform=ax7.transAxes)
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax7.set_ylim([ylim1,ylim2])
    #specify x-axis locations
    x_ticks = x
    #specify x-axis labels
    x_labels = ['Enero', '','Marzo','','Mayo','','Julio','','Septiembre','','Noviembre',''] 
    plt.xticks(ticks=x_ticks, labels=x_labels,fontsize=13)
    #ax7.get_yaxis().set_label_coords(-0.05,0.5)

    plt.text(-0.06, 0.42, "F)", fontweight="bold", fontsize=13 ,transform=ax7.transAxes)



    #FIRST PLOT

    ax1=plt.subplot(611,sharex=ax7)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm1, yerr=dataplotkm1error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km1, yerr=dataplot2km1error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km1, yerr=dataplot3km1error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km1, yerr=dataplot4km1error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax1.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "A)", fontweight="bold", fontsize=13 ,transform=ax1.transAxes)
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.legend()


    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #SECOND PLOT
    ax2=plt.subplot(612,sharex=ax7)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm2, yerr=dataplotkm2error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km2, yerr=dataplot2km2error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km2, yerr=dataplot3km2error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km2, yerr=dataplot4km2error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax2.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "B)", fontweight="bold", fontsize=13 ,transform=ax2.transAxes)
    plt.setp(ax2.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #THIRD PLOT
    ax3=plt.subplot(613,sharex=ax7)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm3, yerr=dataplotkm3error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km3, yerr=dataplot2km3error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km3, yerr=dataplot3km3error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km3, yerr=dataplot4km3error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax3.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "C)", fontweight="bold", fontsize=13 ,transform=ax3.transAxes)
    plt.setp(ax3.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FOURTH PLOT
    ax4=plt.subplot(614,sharex=ax7)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm4, yerr=dataplotkm4error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km4, yerr=dataplot2km4error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km4, yerr=dataplot3km4error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km4, yerr=dataplot4km4error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   

    plt.ylabel('V  (m/s) ',fontsize=13)
    ax4.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "D)", fontweight="bold", fontsize=13 ,transform=ax4.transAxes)
    plt.setp(ax4.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax5=plt.subplot(615,sharex=ax7)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm5, yerr=dataplotkm5error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km5, yerr=dataplot2km5error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km5, yerr=dataplot3km5error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km5, yerr=dataplot4km5error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "E)", fontweight="bold", fontsize=13 ,transform=ax5.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    
    
    
    #------------------------------------------------------------------------------------------------
    
    #PLOT
    ylim1=0
    ylim2=25
    
    
    plt.figure()
    x=np.arange(0,12)
    

    ax12=plt.subplot(6,2,12)

    figsize=(20,4) 
    plt.errorbar(x, dataplotkm12, yerr=dataplotkm12error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km12, yerr=dataplot2km12error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km12, yerr=dataplot3km12error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km12, yerr=dataplot4km12error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    #plt.text(0.01, 0.75, "E)", fontweight="bold", fontsize=13 ,transform=ax7.transAxes)
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax12.set_ylim([ylim1,ylim2])
    #specify x-axis locations
    x_ticks = x
    #specify x-axis labels
    x_labels = ['Enero', '','Marzo','','Mayo','','Julio','','Septiembre','','Noviembre',''] 
    plt.xticks(ticks=x_ticks, labels=x_labels,fontsize=13)
    #ax7.get_yaxis().set_label_coords(-0.05,0.5)

    plt.text(-0.06, 0.42, "L)", fontweight="bold", fontsize=13 ,transform=ax12.transAxes)



    #FIRST PLOT

    ax1=plt.subplot(6,2,1,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm1, yerr=dataplotkm1error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km1, yerr=dataplot2km1error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km1, yerr=dataplot3km1error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km1, yerr=dataplot4km1error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax1.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "A)", fontweight="bold", fontsize=13 ,transform=ax1.transAxes)
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.legend()


    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #SECOND PLOT
    ax2=plt.subplot(6,2,2,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm7, yerr=dataplotkm7error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km7, yerr=dataplot2km7error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km7, yerr=dataplot3km7error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km7, yerr=dataplot4km7error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax2.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "G)", fontweight="bold", fontsize=13 ,transform=ax2.transAxes)
    plt.setp(ax2.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #THIRD PLOT
    ax3=plt.subplot(6,2,3,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm2, yerr=dataplotkm2error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km2, yerr=dataplot2km2error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km2, yerr=dataplot3km2error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km2, yerr=dataplot4km2error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
    
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax3.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "B)", fontweight="bold", fontsize=13 ,transform=ax3.transAxes)
    plt.setp(ax3.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FOURTH PLOT
    ax4=plt.subplot(6,2,4,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm8, yerr=dataplotkm8error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km8, yerr=dataplot2km8error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km8, yerr=dataplot3km8error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km8, yerr=dataplot4km8error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   

    plt.ylabel('V  (m/s) ',fontsize=13)
    ax4.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "H)", fontweight="bold", fontsize=13 ,transform=ax4.transAxes)
    plt.setp(ax4.get_xticklabels(), visible=False)

    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax5=plt.subplot(6,2,5,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm3, yerr=dataplotkm3error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km3, yerr=dataplot2km3error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km3, yerr=dataplot3km3error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km3, yerr=dataplot4km3error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "C)", fontweight="bold", fontsize=13 ,transform=ax5.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax6=plt.subplot(6,2,6,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm9, yerr=dataplotkm9error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km9, yerr=dataplot2km9error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km9, yerr=dataplot3km9error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km9, yerr=dataplot4km9error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "I)", fontweight="bold", fontsize=13 ,transform=ax6.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax7=plt.subplot(6,2,7,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm4, yerr=dataplotkm4error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km4, yerr=dataplot2km4error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km4, yerr=dataplot3km4error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km4, yerr=dataplot4km4error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "D)", fontweight="bold", fontsize=13 ,transform=ax7.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax8=plt.subplot(6,2,8,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm10, yerr=dataplotkm10error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km10, yerr=dataplot2km10error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km10, yerr=dataplot3km10error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km10, yerr=dataplot4km10error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "J)", fontweight="bold", fontsize=13 ,transform=ax8.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax9=plt.subplot(6,2,9,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm5, yerr=dataplotkm5error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km5, yerr=dataplot2km5error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km5, yerr=dataplot3km5error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km5, yerr=dataplot4km5error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "E)", fontweight="bold", fontsize=13 ,transform=ax9.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax10=plt.subplot(6,2,10,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm11, yerr=dataplotkm11error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km11, yerr=dataplot2km11error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km11, yerr=dataplot3km11error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km11, yerr=dataplot4km11error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "K)", fontweight="bold", fontsize=13 ,transform=ax10.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
    #FIFTH PLOT
    ax11=plt.subplot(6,2,11,sharex=ax12)

    figsize=(20,4)
    plt.errorbar(x, dataplotkm6, yerr=dataplotkm6error, linestyle='-', marker='o', capsize=5, label='00:00-06:00')
    plt.errorbar(x, dataplot2km6, yerr=dataplot2km6error, linestyle='-', marker='o', capsize=5, label='06:00-12:00')
    plt.errorbar(x, dataplot3km6, yerr=dataplot3km6error, linestyle='-', marker='o', capsize=5, label='12:00-18:00')
    plt.errorbar(x, dataplot4km6, yerr=dataplot4km6error, linestyle='-', marker='o', capsize=5, label='18:00-24:00')
   
    plt.ylabel('V  (m/s) ',fontsize=13)
    ax5.set_ylim([ylim1,ylim2])
    plt.text(-0.06, 0.42, "F)", fontweight="bold", fontsize=13 ,transform=ax11.transAxes)
    plt.setp(ax5.get_xticklabels(), visible=False)
    
    #ax1.get_yaxis().set_label_coords(-0.05,0.5)
    
   

    
 
    
    
    
    


    
    return

In [10]:
GlobalFunction3('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2018')

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145'

In [8]:
#GlobalFunction('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/4',18,23)

#GlobalFunction2('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/4')
#GlobalFunction2('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2018subsets/4')
GlobalFunction2('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4')

['270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '321', '322', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365']
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/270
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/271
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/272
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/273
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/274
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/275
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets/4/276
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2017subsets